# Plan
<hr>

* Follow up on last weeks exercises
* Functional dependencies
* Keys 
* Normalization
* Cut the crap
* Index - B+ trees
* EAV tables

# Last weeks exercises

## Excercise 1

In the `classicmodels` database, write a query that picks out those customers who are in the same city as office of their sales representative.

### Hand-in:
Insert into the readme file: the query and the graphical execution plan which can be obtained from the query. Explain what is the main performance problem for this query. Do not try to optimize the database for this query (yet).M

## Exercise 2
Change the database schema so that the query from exercise get better performance. 

### Hand-in:
Explain in the readme file what changes you did, if you changed the query or the schema. Insert a new graphical execution plan, and point out in the readme file why this new one is better.

### Exercise 1 & 2
```mysql
select customers.customerName, offices.city as office_city
from customers, employees, offices
where 
	customers.salesRepEmployeeNumber = employees.employeeNumber and 
	employees.officeCode = offices.officeCode and
    customers.city = offices.city;
```

| without index | with index |
|:-------------:|:----------:|
|![](images/Without_city_index.png)|![](images/With_city_index.png)|



## Exercise 3
We want to find out how much each office has sold and the max single payment for each office. Write two queries which give this information

a) using grouping<br>
b) using windowing

For each of the two solutions, check its graphical execution plan.

### Hand-in:
The two queries and the graphical execution plans. Explain any differences and try to explain why there is or is not any difference.

```mysql
WITH bigjoin AS (
	SELECT offices.officeCode AS office, payments.amount as amount
	FROM offices
	INNER JOIN employees ON employees.officeCode = offices.officeCode
	INNER JOIN customers ON customers.salesRepEmployeeNumber = employees.employeeNumber 
	INNER JOIN payments ON payments.customerNumber = customers.customerNumber
)
SELECT office, SUM(amount) as totalSold, MAX(amount) as maxPaid
FROM bigjoin
GROUP BY office
```

![](images/GroupBy.png)

```mysql
WITH bigjoin AS (
	SELECT offices.officeCode as office, payments.amount as amount
	FROM offices
	INNER JOIN employees ON employees.officeCode = offices.officeCode
	INNER JOIN customers ON customers.salesRepEmployeeNumber = employees.employeeNumber 
	INNER JOIN payments ON payments.customerNumber = customers.customerNumber
)
SELECT 
	office, 
	SUM(amount) OVER (PARTITION BY office) AS totalPaid, 
    MAX(amount) OVER (PARTITION BY office) AS maxPaid
from bigjoin
```

## Exercise 4
In the stackexchange forum for coffee (coffee.stackexchange.com), write a query which return the displayName and title of all posts which with the word `grounds`in the title.


### Hand-in:
* Hand in the query. Show the execution plan for the query (if you cannot get the graphical, show the tabular).
* Document that there is no real cost to the join to get the display name instead of just the userid. You can do that by running an other query with no join and then show that there is no major difference.

## Exercise 5
Add a full text index to the `posts` table and change the query from exercise 4 so it no longer scans the entire `posts` table. 

### Hand-in:
* the revised query
* the sql needed to add your index
	* in particular your choice between a "natural language" full-text search and a "boolean" full-text search.
* documentation of efficiency in the form of an execution plan

```mysql
SELECT DisplayName, Title
FROM posts
INNER JOIN users ON OwnerUserId = users.Id
WHERE MATCH(Title) AGAINST ('grounds' IN BOOLEAN MODE)
```
![](images/FulltextIndex.png)

# Todays program

* Functional dependencies
* Keys 
* Normalization
* Cut the crap
* Index - B+ trees
* EAV tables

# What is this relation thing again???
Consider this table:

![](images/Normal_1.png)

A **relation** is a **subset** of the **crossproduct** of the column types.

$R = D_{0} \times D_{1} \times D_{2} \times ... \times D_{n}$

Or the above: $ClientRental = String \times String \times String \times String \times Date \times Date \times Int \times String \times String$

# A relation is a subset of the crossproduct of the column types.

Each row in a table is an element (tuple) from the relation.

In the mathematical form: $row \in R$ 

or written out: $(v_{0},v_{1},...,v_{n}) \in D_{0} \times D_{1} \times D_{2} \times ... \times D_{n}$

in our ClientRental example:
$("CR76","PG4","John\:Kay", "6\:Lawrence\:St,\:Glassgow", 1-jul-03, 31-aug-04,350, "CO40", "Tina\: Murphy") \in ClientRental$

##  What is a function

A function is a mapping from one domain to an other.

For example $age: Person \rightarrow Integer$. 
Type $Integer$ is sometimes written ℕ

A function maps from its domain. The Codomain and Range are both on the output side, but are subtly different. The Codomain is the set of values that could possibly come out. ... And The Range is the set of values that actually do come out

For example the **codomain** of the *age* function is ℕ, but the **range** is $[0..120]$ (more or less)

## Dependency

If there is a function $f: A \rightarrow B$ from column $A$ to column $B$, we say that $B$ is **(functionally) dependent** on $A$.

We write it as: $A \rightarrow B$

Mostly such functions are not "mathematically", but has to do with the world we model. 
* PostalCodes determine Cities
* emails determine Persons
* product determines price
* ...



### Multicolumn functions and dependencies
Notice: A function can map from column A and B to C. $\:\:f: A \times B \rightarrow C$.

Hence the dependency is:  $A \times B \rightarrow C$

## Primary key, candidate key

A **candicate Key** is a minimal subset of columns that functionally determines the remaining collumns.

Assume we have a relation $R = D_{0} \times D_{1} \times D_{2} \times ... \times D_{n}$

$C_{xyz}$ (columns X,Y and Z) is a candidate key if $C_{xyz} \rightarrow D_0, C_{xyz} \rightarrow D_1, C_{xyz} \rightarrow D_2, ... C_{xyz} \rightarrow D_n$.

A column is always dependent on itself: $C \rightarrow C$.

A relation do not **have** a primary key, we as designers pick one candidate key and decide it to be **primary key**.

## First normal form

### All tables must have a primary key
### All columns are a single domain.

Relation to real world needed to determine this. 

* Full name vs. first and last. 
* Adressline vs. streetname and number (and floor and door).
* Country/Area code in phone numbers

The support for json and xml violates this principle. The query language now must have ways to extract information from inside a structured value.



## Second normal form

### The table must be on first normal form

### All the non-key columns are  dependent on all of the table’s primary key.

Only of interest if you have composite keys $A \times B$. In that case you can have a dependecy $B \rightarrow C$.

![](images/Normal_2.png)

Primary key: $StudentId \times CourseId$

Dependencies: 
-  $StudentId \rightarrow StuName$ 
-  $CourseId \rightarrow CourseName$, 
-  $StudentId \times CourseId \rightarrow Grade$

## Many to Many relationships

The classic example is orderlines which links orders and products. In the classicmodels database

```mysql
create table orderdetails (
  orderNumber     int            not null,
  productCode     varchar(15)    not null,
  quantityOrdered int            not null,
  priceEach       decimal(10, 2) not null,
  orderLineNumber smallint(6)    not null)
```
Is there a dependency $productCode \rightarrow priceEach$ ?

The most common reason for violations of 2<sup>nd</sup> normal form is attributes in a many to many relations which really belong to one of the connected tables.

## Third normal form

### 2<sup>nd</sup> normal form
### No *transitive dependencies*
A transitive dependency is when a non-key attribute depends on another non-key attribute.

In math speak. If there is a dependency $A \rightarrow B$ in the table, then A must be the key. Or $A \times B \rightarrow C$

### 1<sup>st</sup>, 2<sup>nd</sup> and 3<sup>rd</sup>
Bill Kent: "[Every] non-key [attribute] must provide a fact about the key, the whole key, and nothing but the key."



## Higher order
Boyce-Codd, Fourth, and I want my name on a paper too....

# Summary of normal forms
* Give the table a "surrogate key" (a key which is not from the domain, but an auto incrementing integer)
* Make sure the columns only deal with the domain entity
* Take care with many-many relationships

# Your turn

![](images/InclassExercise7_1.png)

# Exercise until you get it...

# Break

# Index - B+ trees

A binary tree has two subtrees. A B-tree use the size of the disc sector as limit on the children. 

If we index a column of integers (8 bytes), and the sector size is 4K, we can have 4096 / 8 = 512 numbers. 

The actual tuples are not in the tree (we could have more than one index over the data). We will need reserve some space for the tuple index as well (an other long). 

For integer indexes - the branch out could therefore be around 256.

Assume we have 500.000.000 records (about the population of EU). What is the height of the tree?

$log_{256}(500.000.000) = log_{10}(500.000.000)/log_{10}(256) = 8.67/2.41 = 3.6 \rightarrow 4$ (compared to 29 for a binary tree).

## Leaf nodes are linked list

Linked lists are efficient for insertion, removal, iteration, and spans

<img src="images/Bplus_LeafNodes.png" width="50%">


# The B+tree

<img src="images/Bplus_BranchNodes.png" width="50%">

## B tree search

Target: 57

<img src="images/Bplus_Search.png" width="50%">

## What can we do with the index?
* Sorting?
* updates?
* inserts?
* Which where clauses can we handle?
* like?

## Like on text index
```mysql
select *
from customers
where customerName like 'Mini%'
```
What can it do?

## Like continued
<img src="images/Bplus_Like.png" width="50%">

### Try to avoid '%' in the beginning
Why?

## What is the cost of "adding an other index"

|Insert|Delete|
|:------:|:------:|
|<img src="images/Bplus_index_cost.png">|<img src="images/Bplus_delete_cost.png">|



# And now to something completely different

# Entity-attribute-value tables

One table to rule them all


|Definition|Data|
|:------:|:------:|
|<img src="images/EAV_table.png">|<img src="images/EAV_table_data.png">|


### finding all customers in the USA

regular sql:

```sql
select customerNumber, customerName,city, country
from customers
where country='USA'
```
compare with

```sql
with 
     usa_companies as (
       select customerNumber as id
       from eav
       where attribute = 'country'
         and value = 'USA')
select *
from eav
where eav.customerNumber in (select * from usa_companies)
```

## Entity-attribute-value tables
Extract them from existing tables

```sql
create table eav as
  select customerNumber, 'id' as attribute, customerNumber as value
  from customers
  union
  select customerNumber, 'customerName' as attribute, customerName as value
  from customers
  union
  select customerNumber, 'phone' as attribute, phone as value
  from customers
  union
  select customerNumber, 'city' as attribute, city as value
  from customers
  union
  select customerNumber, 'country' as attribute, country as value
  from customers
```







### WARNING: Meta programming in SQL

```sql
with cols as
   (SELECT concat('(select concat(\'customers_\',customerNumber) as id, \'', COLUMN_NAME,
     '\' as attr, ', COLUMN_NAME, ' as value)') as query
    FROM information_schema.columns
    WHERE table_schema = 'classicmodels'
      AND table_name = 'customers')
select group_concat(query separator ' union ')
from cols
```

which yields:

```mysql
(select concat('customers_', customerNumber) as id, 'customerNumber' as attr, customerNumber as value from customers)
union
(select concat('customers_', customerNumber) as id, 'customerName' as attr, customerName as value from customers)
union
(select concat('customers_', customerNumber) as id, 'contactLastName' as attr, contactLastName as value from customers)
union
(select concat('customers_', customerNumber) as id, 'contactFirstName' as attr, contactFirstName as value
 from customers)
union
(select concat('customers_', customerNumber) as id, 'phone' as attr, phone as value from customers)
union
(select concat('customers_', customerNumber) as id, 'addressLine1' as attr, addressLine1 as value from customers)
union
(select concat('customers_', customerNumber) as id, 'addressLine2' as attr, addressLine2 as value from customers)
union
(select concat('customers_', customerNumber) as id, 'city' as attr, city as value from customers)
...
```

# Assignments for next time